# VLB Creep Compliance: NLSQ → NUTS

**Polystyrene at 190 °C — creep compliance J(t)**

## Learning Objectives

- Fit creep compliance J(t) to extract G₀ and k_d
- Understand Maxwell creep: J(t) = (1/G₀)(1 + k_d·t)
- Compare NLSQ and NUTS posteriors for both parameters

## Data Source

Polystyrene creep at 190 °C from pyRheo demo datasets.

## Estimated Runtime

- Fast demo (1 chain): ~1 min
- Full run (4 chains): ~3 min

In [ ]:
import sys
IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax openpyxl
    import os
    os.environ["JAX_ENABLE_X64"] = "true"

In [ ]:
%matplotlib inline
import warnings
import time
from pathlib import Path

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models import VLBLocal

jax, jnp = safe_import_jax()
verify_float64()
warnings.filterwarnings('ignore', category=FutureWarning)

# Tutorial utilities
import sys
sys.path.insert(0, str(Path('..').resolve()))
from utils.vlb_tutorial_utils import (
    get_bayesian_config, print_convergence, print_parameter_table,
    plot_trace_and_forest, get_output_dir, save_results, save_figure,
    load_ps_creep,
)

print(f"JAX version: {jax.__version__}")

## 2. Load Experimental Data

In [ ]:
time_data, J_data = load_ps_creep(190, max_points=200)

print(f'Data points: {len(time_data)}')
print(f'Time range: {time_data.min():.2f} - {time_data.max():.0f} s')
print(f'J range: {J_data.min():.2e} - {J_data.max():.2e} 1/Pa')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

ax1.plot(time_data, J_data, 'ko', markersize=3)
ax1.set_xlabel('Time [s]')
ax1.set_ylabel('J(t) [1/Pa]')
ax1.set_title('Creep Compliance')
ax1.grid(True, alpha=0.3)

ax2.loglog(time_data, J_data, 'ko', markersize=3)
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('J(t) [1/Pa]')
ax2.set_title('Creep Compliance (log-log)')
ax2.grid(True, alpha=0.3, which='both')

plt.tight_layout()
display(fig)
plt.close(fig)

## 3. VLB Forward Model

For a Maxwell fluid under constant stress $\sigma_0$:

$$J(t) = \frac{1}{G_0}(1 + k_d \cdot t)$$

- **Short times:** elastic response $J(0) = 1/G_0$
- **Long times:** viscous flow $J \to k_d \cdot t / G_0 = t/\eta_0$
- Both $G_0$ and $k_d$ are individually identifiable from creep data

We set `sigma_applied=1.0` so the model returns $J(t)$ directly.

## 4. Step 1: NLSQ Point Estimation

In [ ]:
model = VLBLocal()

t0 = time.time()
model.fit(time_data, J_data, test_mode='creep', sigma_applied=1.0)
t_nlsq = time.time() - t0

print(f'NLSQ fit time: {t_nlsq:.2f} s')
print(f'G0  = {model.G0:.2e} Pa')
print(f'k_d = {model.k_d:.4e} 1/s')
print(f'eta_0 = {model.G0 / model.k_d:.2e} Pa*s')

# Fit quality
time_fine = np.linspace(time_data.min(), time_data.max(), 300)
J_pred_fine = model.predict(time_fine, test_mode='creep', sigma_applied=1.0)
J_pred = model.predict(time_data, test_mode='creep', sigma_applied=1.0)
ss_res = np.sum((J_data - np.array(J_pred))**2)
ss_tot = np.sum((J_data - np.mean(J_data))**2)
r2 = 1 - ss_res / ss_tot
print(f'R-squared = {r2:.6f}')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

ax1.plot(time_data, J_data, 'ko', markersize=3, label='Data')
ax1.plot(time_fine, J_pred_fine, 'r-', lw=2, label='VLB fit')
ax1.set_xlabel('Time [s]')
ax1.set_ylabel('J(t) [1/Pa]')
ax1.set_title('Creep Compliance Fit')
ax1.legend()
ax1.grid(True, alpha=0.3)

residuals = (J_data - np.array(J_pred)) / J_data * 100
ax2.plot(time_data, residuals, 'ko-', markersize=3)
ax2.axhline(0, color='gray', ls='--')
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Relative residual [%]')
ax2.set_title('Residuals')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

## 4. Step 2: Bayesian Inference (NUTS)

In [ ]:
config = get_bayesian_config()
initial_values = {'G0': float(model.G0), 'k_d': float(model.k_d)}
print(f'Config: {config}')
print(f'Warm-start: {initial_values}')

t0 = time.time()
result = model.fit_bayesian(
    time_data, J_data, test_mode='creep',
    sigma_applied=1.0,
    initial_values=initial_values,
    seed=42,
    **config,
)
t_bayes = time.time() - t0
print(f'\nBayesian inference time: {t_bayes:.1f} s')

## 5. Convergence Diagnostics

In [ ]:
param_names = ["G0", "k_d"]
converged = print_convergence(result, param_names)

In [ ]:
fig_trace, fig_forest = plot_trace_and_forest(result, param_names)
display(fig_trace)
plt.close(fig_trace)
display(fig_forest)
plt.close(fig_forest)

## 6. Posterior Summary

In [ ]:
posterior = result.posterior_samples
nlsq_vals = {"G0": model.G0, "k_d": model.k_d}
print_parameter_table(["G0", "k_d"], nlsq_vals, posterior)

## 7. Posterior Predictive Check

In [ ]:
posterior = result.posterior_samples
n_draws = min(200, len(posterior['G0']))

J_samples = []
for i in range(n_draws):
    model.parameters.set_value('G0', float(posterior['G0'][i]))
    model.parameters.set_value('k_d', float(posterior['k_d'][i]))
    J_samples.append(np.array(
        model.predict(time_fine, test_mode='creep', sigma_applied=1.0)
    ))

J_arr = np.array(J_samples)

fig, ax = plt.subplots(figsize=(9, 6))
ax.fill_between(time_fine,
    np.percentile(J_arr, 2.5, axis=0),
    np.percentile(J_arr, 97.5, axis=0),
    alpha=0.3, color='C0', label='95% CI')
ax.plot(time_fine, np.median(J_arr, axis=0),
        'C0-', lw=2, label='Posterior median')
ax.plot(time_data, J_data, 'ko', markersize=3, label='Data')
ax.set_xlabel('Time [s]')
ax.set_ylabel('J(t) [1/Pa]')
ax.set_title('Posterior Predictive Check')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)
plt.tight_layout()
display(fig)
plt.close(fig)

## 8. Save Results

In [ ]:
output_dir = get_output_dir("creep")
save_results(output_dir, model, result, ["G0", "k_d"], None)
print("Done.")

## Key Takeaways

1. **Creep compliance identifies both G₀ and k_d** independently
2. **Linear J(t) at long times** confirms Maxwell viscous flow
3. **Short-time intercept** gives 1/G₀ directly
4. **Long-time slope** gives k_d/G₀ = 1/η₀

## Next

- **Notebook 13**: Stress relaxation NLSQ → NUTS